# Plag-Liq and Kspar-Liq Matching
- This notebook shows how to assess all possible matches for Plag-Liq and Kspar-Liq pairs where there aren't obvious pairwise analyses
- You can download the excel spreadsheet here: https://github.com/PennyWieser/Thermobar/blob/main/docs/Examples/Feldspar_Thermobarometry/Feldspar_Liquid.xlsx

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Thermobar as pt
pd.options.display.max_columns = None

## Example 1 - Plag and Liquid. 

In [2]:
## Load in the liquids
out_Liq=pt.import_excel('Feldspar_Liquid.xlsx', sheet_name="Liq_only")
Liqs=out_Liq['Liqs']
# Load in the Plags
out_Plag=pt.import_excel('Feldspar_Liquid.xlsx', sheet_name="Plag_only")
Plags=out_Plag['Plags']

In [3]:
## Inspect them to check they read in right
display(Plags.head())
display(Liqs.head())

,SiO2_Plag,TiO2_Plag,Al2O3_Plag,FeOt_Plag,MnO_Plag,MgO_Plag,CaO_Plag,Na2O_Plag,K2O_Plag,Cr2O3_Plag,Sample_ID_Plag
0,57.3,0.09,26.6,0.43,0.0,0.03,8.33,6.11,0.49,0.0,Plag1
1,56.5,0.12,26.9,0.47,0.0,0.05,8.95,5.66,0.47,0.0,Plag2
2,57.6,0.11,26.3,0.50,0.0,0.07,8.50,6.27,0.40,0.0,Plag3_core
3,57.2,0.16,27.0,0.62,0.0,0.06,9.03,5.58,0.84,0.0,Plag4
4,56.7,0.14,27.6,0.69,0.0,0.11,9.46,5.58,0.48,0.0,Plag5


,SiO2_Liq,TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,CaO_Liq,Na2O_Liq,K2O_Liq,Cr2O3_Liq,P2O5_Liq,H2O_Liq,Fe3Fet_Liq,NiO_Liq,CoO_Liq,CO2_Liq,Sample_ID_Liq
0,49.1,3.22,14.4,14.8,0.14,3.20,6.72,3.34,1.70,0.0,1.13,0.0,0.0,0.0,0.0,0.0,Glass1
1,49.2,3.89,15.3,13.7,0.12,3.88,6.76,3.44,1.22,0.0,0.83,0.0,0.0,0.0,0.0,0.0,Glass2
2,49.6,3.79,15.8,13.0,0.14,4.26,6.59,3.65,1.04,0.0,0.63,0.0,0.0,0.0,0.0,0.0,Glass3
3,47.1,4.21,12.0,17.8,0.18,3.40,7.28,2.93,2.02,0.0,2.32,0.0,0.0,0.0,0.0,0.0,Glass4
4,48.1,3.88,13.2,16.4,0.16,4.02,6.51,3.36,1.36,0.0,1.59,0.0,0.0,0.0,0.0,0.0,Glass5


## Example 1a - All possible Liq-Plag matches, An-Ab equilibrium
- You can specify to use the An-Ab equilibrium test of Putirka (2008) as in this example

In [4]:
MM_dict=pt.calculate_fspar_liq_temp_matching(liq_comps=Liqs, plag_comps=Plags, 
                                             equationT="T_Put2008_eq24a", P=5, 
                                                Ab_An_P2008=True, H2O_Liq=2)
Av_Matches=MM_dict['Av_PTs']
All_Matches=MM_dict['All_PTs']

Considering N=28 Fspar & N=24 Liqs, which is a total of N=672 Liq-Fspar pairs, be patient if this is >>1 million!
Done!!! I found a total of N=336 Fspar-Liq matches using the specified filter. N=28 Fspar out of the N=28 Fspar that you input matched to 1 or more liquids


## Example 1b - All possible Liq-Plag matches, no filter
- This could be used if you want to develop your own equilibrium filters

In [5]:
MM_dict=pt.calculate_fspar_liq_temp_matching(liq_comps=Liqs, plag_comps=Plags, 
                                             equationT="T_Put2008_eq24a", P=5, 
                                                Ab_An_P2008=False, H2O_Liq=2)
Av_Matches=MM_dict['Av_PTs']
All_Matches=MM_dict['All_PTs']

Considering N=28 Fspar & N=24 Liqs, which is a total of N=672 Liq-Fspar pairs, be patient if this is >>1 million!
We are returning all pairs, if you want to use the Ab-An equilibrium test of Putirka (2008), enter Ab_An_P2008=True
Done!!! I found a total of N=672 Fspar-Liq matches using the specified filter. N=28 Fspar out of the N=28 Fspar that you input matched to 1 or more liquids


## Example 1c - Iterating T and H2O for all possible Plag-Liq matches

In [9]:
T_H_Iter_dict=pt.calculate_fspar_liq_temp_hygr_matching(liq_comps=Liqs, plag_comps=Plags,
        equationT="T_Put2008_eq24a", equationH="H_Waters2015", P=5)
T_H_Calc_Av=T_H_Iter_dict.get('Av_HTs')
T_H_Calc_All=T_H_Iter_dict.get('All_HTs')
T_H_Evol=T_H_Iter_dict.get('T_H_Evolution')

Considering N=28 Fspar & N=24 Liqs, which is a total of N=672 Liq- Fspar pairs, be patient if this is >>1 million!
20
Done!!! I found a total of N=672 Fspar-Liq matches using the specified filter. N=28 Fspar out of the N=28 Fspar that you input matched to 1 or more liquids


In [10]:
T_H_Calc_Av

,ID_Fspar,Mean_Sample_ID_Plag,Mean_# of iterations,Mean_Delta T (last 2 iters),Mean_H2O_calc,Mean_Delta H (last 2 iters),Mean_Delta_An,Mean_Delta_Ab,Mean_Delta_Or,Mean_Pred_An_EqE,Mean_Pred_Ab_EqF,Mean_Pred_Or_EqG,Mean_Obs_Kd_Ab_An,Mean_Si_Plag_cat_prop,Mean_Mg_Plag_cat_prop,Mean_Fet_Plag_cat_prop,Mean_Ca_Plag_cat_prop,Mean_Al_Plag_cat_prop,Mean_Na_Plag_cat_prop,Mean_K_Plag_cat_prop,Mean_Mn_Plag_cat_prop,Mean_Ti_Plag_cat_prop,Mean_Cr_Plag_cat_prop,Mean_sum,Mean_Si_Plag_cat_frac,Mean_Mg_Plag_cat_frac,Mean_Fet_Plag_cat_frac,Mean_Ca_Plag_cat_frac,Mean_Al_Plag_cat_frac,Mean_Na_Plag_cat_frac,Mean_K_Plag_cat_frac,Mean_Mn_Plag_cat_frac,Mean_Ti_Plag_cat_frac,Mean_Cr_Plag_cat_frac,Mean_An_Plag,Mean_Ab_Plag,Mean_Or_Plag,Mean_SiO2_Liq,Mean_TiO2_Liq,Mean_Al2O3_Liq,Mean_FeOt_Liq,Mean_MnO_Liq,Mean_MgO_Liq,Mean_CaO_Liq,Mean_Na2O_Liq,Mean_K2O_Liq,Mean_Cr2O3_Liq,Mean_P2O5_Liq,Mean_H2O_Liq,Mean_Fe3Fet_Liq,Mean_NiO_Liq,Mean_CoO_Liq,Mean_CO2_Liq,Mean_ID_liq,Mean_Sample_ID_liq,Mean_SiO2_Liq_mol_frac,Mean_MgO_Liq_mol_frac,Mean_MnO_Liq_mol_frac,Mean_FeOt_Liq_mol_frac,Mean_CaO_Liq_mol_frac,Mean_Al2O3_Liq_mol_frac,Mean_Na2O_Liq_mol_frac,Mean_K2O_Liq_mol_frac,Mean_TiO2_Liq_mol_frac,Mean_P2O5_Liq_mol_frac,Mean_Cr2O3_Liq_mol_frac,Mean_Si_Liq_cat_frac,Mean_Mg_Liq_cat_frac,Mean_Mn_Liq_cat_frac,Mean_Fet_Liq_cat_frac,Mean_Ca_Liq_cat_frac,Mean_Al_Liq_cat_frac,Mean_Na_Liq_cat_frac,Mean_K_Liq_cat_frac,Mean_Ti_Liq_cat_frac,Mean_P_Liq_cat_frac,Mean_Cr_Liq_cat_frac,Mean_Mg_Number_Liq_NoFe3,Mean_Mg_Number_Liq_Fe3,Mean_P,Mean_T,Mean_T_K_calc,# of Liqs Averaged,Std_Sample_ID_Plag,Std_# of iterations,Std_Delta T (last 2 iters),Std_H2O_calc,Std_Delta H (last 2 iters),Std_Delta_An,Std_Delta_Ab,Std_Delta_Or,Std_Pred_An_EqE,Std_Pred_Ab_EqF,Std_Pred_Or_EqG,Std_Obs_Kd_Ab_An,Std_Si_Plag_cat_prop,Std_Mg_Plag_cat_prop,Std_Fet_Plag_cat_prop,Std_Ca_Plag_cat_prop,Std_Al_Plag_cat_prop,Std_Na_Plag_cat_prop,Std_K_Plag_cat_prop,Std_Mn_Plag_cat_prop,Std_Ti_Plag_cat_prop,Std_Cr_Plag_cat_prop,Std_sum,Std_Si_Plag_cat_frac,Std_Mg_Plag_cat_frac,Std_Fet_Plag_cat_frac,Std_Ca_Plag_cat_frac,Std_Al_Plag_cat_frac,Std_Na_Plag_cat_frac,Std_K_Plag_cat_frac,Std_Mn_Plag_cat_frac,Std_Ti_Plag_cat_frac,Std_Cr_Plag_cat_frac,Std_An_Plag,Std_Ab_Plag,Std_Or_Plag,Std_SiO2_Liq,Std_TiO2_Liq,Std_Al2O3_Liq,Std_FeOt_Liq,Std_MnO_Liq,Std_MgO_Liq,Std_CaO_Liq,Std_Na2O_Liq,Std_K2O_Liq,Std_Cr2O3_Liq,Std_P2O5_Liq,Std_H2O_Liq,Std_Fe3Fet_Liq,Std_NiO_Liq,Std_CoO_Liq,Std_CO2_Liq,Std_ID_liq,Std_Sample_ID_liq,Std_SiO2_Liq_mol_frac,Std_MgO_Liq_mol_frac,Std_MnO_Liq_mol_frac,Std_FeOt_Liq_mol_frac,Std_CaO_Liq_mol_frac,Std_Al2O3_Liq_mol_frac,Std_Na2O_Liq_mol_frac,Std_K2O_Liq_mol_frac,Std_TiO2_Liq_mol_frac,Std_P2O5_Liq_mol_frac,Std_Cr2O3_Liq_mol_frac,Std_Si_Liq_cat_frac,Std_Mg_Liq_cat_frac,Std_Mn_Liq_cat_frac,Std_Fet_Liq_cat_frac,Std_Ca_Liq_cat_frac,Std_Al_Liq_cat_frac,Std_Na_Liq_cat_frac,Std_K_Liq_cat_frac,Std_Ti_Liq_cat_frac,Std_P_Liq_cat_frac,Std_Cr_Liq_cat_frac,Std_Mg_Number_Liq_NoFe3,Std_Mg_Number_Liq_Fe3,Std_P,Std_T,Std_T_K_calc,Sample_ID_Plag
0,0,Plag1,20.0,0.000009,-0.152865,-2.516342e-07,0.156529,0.164705,0.028989,0.573657,0.388952,0.000226,0.687211,0.953660,0.000744,0.005985,0.148545,0.521768,0.197164,0.010404,0.000000,0.001127,0.0,1.839397,0.518464,0.000405,0.003254,0.080757,0.283663,0.107189,0.005656,0.000000,0.000613,0.0,0.417129,0.553656,0.029215,49.082083,3.40875,14.079167,13.005,0.180833,4.797917,8.408333,3.212083,1.243333,0.0,0.782917,0.0925,0.0,0.0,0.0,0.0,11.5,11.5,0.53705,0.078143,0.001675,0.119259,0.098482,0.090789,0.034093,0.008698,0.028164,0.003648,0.0,0.472184,0.068771,0.001474,0.10486,0.08669,0.159639,0.059923,0.015287,0.024757,0.006414,0.0,0.395638,0.395638,5.0,1417.824824,1417.824834,24,Plag1,0.0,0.000035,0.575867,9.742417e-07,0.092707,0.091742,0.000277,0.092707,0.091742,0.000277,0.280843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.680864,1.171897,1.065274,2.169584,0.036584,1.213167,1.941711,0.394098,0.443442,0.0,0.480756,0.124035,0.0,0.0,0.0,0.0,7.071

In [ ]:
    FsparNumbers = Combo_fspar_liqs2['ID_Fspar'].unique()

    Combo_fspar_liqs3=Combo_fspar_liqs2.drop(['Pass An-Ab Eq Test Put2008?', 'T_K_calc'], axis=1)
    Combo_fspar_liqs3['T_K_calc']=Combo_fspar_liqs2['T_K_calc'].astype(float)
    if len(FsparNumbers) > 0:
        if plag_comps is not None:
            df1_Mean_nopref=Combo_fspar_liqs3.groupby(['ID_Fspar', 'Sample_ID_Plag'], as_index=False).mean()
            df1_Std_nopref=Combo_fspar_liqs3.groupby(['ID_Fspar', 'Sample_ID_Plag'], as_index=False).std()
            count=Combo_fspar_liqs2.groupby('ID_Fspar',as_index=False).count().iloc[:, 1]
            df1_Mean_nopref['# of Liqs Averaged']=count

            Sample_ID_Fspar_Mean=df1_Mean_nopref['Sample_ID_Plag']
            Sample_ID_Fspar_Std=df1_Std_nopref['Sample_ID_Plag']
            df1_Mean=df1_Mean_nopref.add_prefix('Mean_')
            df1_Std=df1_Std_nopref.add_prefix('Std_')

            df1_Mean.rename(columns={"Mean_ID_Fspar": "ID_Fspar"}, inplace=True)
            df1_Mean.rename(columns={"Mean_# of Liqs Averaged": "# of Liqs Averaged"}, inplace=True)
            df1_Std.rename(columns={"Std_ID_Fspar": "ID_Fspar"}, inplace=True)



            df1_M=pd.merge(df1_Mean, df1_Std, on=['ID_Fspar'])
            df1_M['Sample_ID_Plag']=Sample_ID_Fspar_Mean

    else:
        raise Exception(
            'No Matches - to set less strict filters, change our Kd filter')


    print('Done!!! I found a total of N='+str(len(Combo_fspar_liqs3)) + ' Fspar-Liq matches using the specified filter. N=' + str(len(df1_M)) + ' Fspar out of the N='+str(LenFspar)+' Fspar that you input matched to 1 or more liquids')

    return {'Av_PTs': df1_M, 'All_PTs': Combo_fspar_liqs2}
    # Now we do the averaging step for each feldspar crystal




### Again, lets filter using the equilibrium test of Putirka (2008)

In [ ]:
T_H_Calc_Filter=T_H_Calc.loc[T_H_Calc['Pass An-Ab Eq Test Put2008?'].str.contains('Yes')]

In [ ]:
# Maybe you want to plot a histogram of H2O contents
plt.hist(T_H_Calc_Filter['H2O_calc'])
plt.xlabel('H$_2$O Calc (wt%)')

## Example 2 - Kspar-Liquid calculations

In [ ]:
# Load in the Kspars
out_Kspar=pt.import_excel('Feldspar_Liquid.xlsx', sheet_name="Kspar_only")
Kspars=out_Kspar['Kspars']

# Load in the liquids you want to match with
out_Liq2=pt.import_excel('Feldspar_Liquid.xlsx', sheet_name="Evolved_Liq_only")
Liqs=out_Liq2['Liqs']

### 2a - Lets calculate temperature for all liq-kspar matches
- Currently no equilibrium tests exist for Kspar-Liquid that we are aware off. If you know of ones I can implement, please reach out!

In [ ]:
melt_match_2=pt.calculate_fspar_liq_temp_matching(liq_comps=Liqs, kspar_comps=Kspars, 
                                             equationT="T_Put2008_eq24b", P=5, 
                                                eq_tests=True, H2O_Liq=2)
melt_match_2